In [2]:
import diffusers
import torch
from diffusers import (
    AutoencoderKL,
    ControlNetModel,
    DDPMScheduler,
    StableDiffusionXLControlNetPipeline,
    UNet2DConditionModel,
    UniPCMultistepScheduler,
)


In [3]:
pretrained_model_name_or_path = "/mnt/workspace/workgroup_share/lhn/models/runwayml/stable-diffusion-v1-5"
weight_dtype = torch.bfloat16
device = "cuda"
prompt = 'photorealistic, ultra-detailed, cinematic photo, raw photo, highrek, 4k, professional, ultra-detailed, hires, 8k, Super Detail,masterpiece,best quality,highly detailed,good compositon,' 
variant = None
revision = None
unet = UNet2DConditionModel.from_pretrained(
    pretrained_model_name_or_path, subfolder="unet", revision=revision, variant=variant
)
vae_path = pretrained_model_name_or_path
# pretrained_vae_model_name_or_path = "/mnt/workspace/workgroup_share/lhn/models/sd-vae-ft-mse"
noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")
vae = AutoencoderKL.from_pretrained(
    vae_path,
    subfolder="vae",
    revision=revision,
    variant=variant,
)
vae.to(device,dtype=torch.float32)
unet.to(device, dtype=weight_dtype)

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): LoRACompatibleLinear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): LoRACompatibleLinear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): LoRACompatibleConv(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): LoRACompatibleLinear(in_features=320, out_features=320, bias=False)
                (to_k): LoRACompatibleLinear(in_features=320, out_features=320

In [4]:
batch = {}
batch["pixel_values"] = torch.randn((1,3,512,512))
batch["prompt_ids"] = torch.randn((1,77,1024))

In [7]:
noisy_latents.shape

torch.Size([1, 4, 64, 64])

In [5]:

# Convert images to latent space
pixel_values = batch["pixel_values"].to(device,dtype=torch.float32)
# vae.to(device,dtype=torch.float32)
latents = vae.encode(pixel_values).latent_dist.sample()
latents = latents * vae.config.scaling_factor

# Sample noise that we'll add to the latents
noise = torch.randn_like(latents)
bsz = latents.shape[0]

# Sample a random timestep for each image
timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
timesteps = timesteps.long()

# Add noise to the latents according to the noise magnitude at each timestep
# (this is the forward diffusion process)
noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

# Predict the noise residual
model_pred = unet(
    noisy_latents,
    timesteps,
    encoder_hidden_states=batch["prompt_ids"],
).sample

RuntimeError: mat1 and mat2 must have the same dtype